shift +cmd+ i to bring the copilot chat

Ornstein-Uhlenbeck (OU) process:

It is a popular stochastic differential equation (SDE) to model mean-reverting spreads between two correlated assets.

Before fitting an OU process, ensure that the two assets are cointegrated (i.e., their spread is mean-reverting).

Common methods:
Engle-Granger Test (ADF test on residuals of a linear regression).
Johansen Test (for multiple assets).

$$
dc_t = \kappa (\theta - c_t)\,dt + \sigma\,dW_t
$$


Model the spread as a discrete-time OU process:
$$
\text{Spread}_{t+1} = a + b \cdot \text{Spread}_t + \eta_t
$$
Estimate parameters a and b using linear regression of 
$$
\text{Spread}_{t+1} on  

\text{Spread}_t. 
$$
Then, compute the OU parameters:
	•	Mean reversion rate (κ): 
    $$
    \kappa = -\ln(b) / \Delta t
    $$
	•	Long-term mean (θ): 
    $$
    \theta = a / (1 - b)
    $$
	•	Volatility (σ): 
    $$
    \sigma = \text{std}(\eta_t) \cdot \sqrt{2\kappa / (1 - b^2)}
    $$
Assume 
$$
\Delta t = 1/252 
$$
for daily data.

When ADF P value =0.9499, it is much greate than 0.05, meaning we fail to reject null hypotheis, that time series has a unit root, i.e. it is non-stationary.

stationary implies constant mean and variance, then we can say two stock are cointegrated

Half - life=ln(2)/k, for instance, if k=74.43 for the data by minute, then half - life = 0.0093 years, or 0.0093 * (252 days) * (390 min) = 914 minutes, or 2.3 trading day




$$
\frac{F_-(e)}{F_-’(e)} = \frac{H^+(e) - e - c}{H^+’(e) - 1}
$$


Great! Let’s walk through a visual and mathematical explanation of where the optimal entry threshold condition comes from — using stochastic optimal stopping theory.

We’ll work step by step toward the equation:
$$
\frac{F_-(e)}{F_-’(e)} = \frac{H^+(e) - e - c}{H^+’(e) - 1}
$$
⸻

🧠 Setup: Two-Level Optimal Stopping Problem

We want to find the optimal time to enter a long position in a mean-reverting spread E_t, modeled by an Ornstein–Uhlenbeck (OU) process:
$$
dE_t = \kappa(\theta - E_t)dt + \sigma dW_t
$$
There are two value functions:

🔹 1. Exit Value Function H^+(e)

Once you enter the trade, your position has value:
	•	If the spread rises and hits the exit threshold 
	$$
	E^*_{\text{exit}}
	$$
	, you close out and receive E_t - c
	•	If it hasn’t yet, you wait — and the value decays based on expected time to exit

🔹 2. Entry Value Function G^+(e)

Before entering the trade, the agent chooses when to act. The value function is:
$$
G^+(e) = \sup_\tau \mathbb{E}e \left[ e^{-\rho \tau} \left(H^+(E\tau) - E_\tau - c \right) \right]
$$
•	You pay 
$$
E_\tau + c (spread + transaction cost)
$$
•	You receive value 
$$
H^+(E_\tau) if you enter
$$
•	You discount at rate 
$$
\rho
$$
⸻

✏️ The Entry Problem: Smooth Pasting Condition

In optimal stopping theory, the optimal entry threshold 

$$E^*_{\text{entry}}
$$
 satisfies:
	1.	Value matching: 
	$$
	G^+(e) = H^+(e) - e - c
	$$
	2.	Smooth pasting: 
	$$
	\frac{dG^+}{de} = H^+’(e) - 1
	$$
This ensures the value function is smooth at the boundary, avoiding arbitrage or sudden jumps in value. These two conditions are required for continuous fit and first-order optimality.

⸻

🔧 Analytical Tools: Fundamental Solutions

To solve the entry problem, we use:
$$
G^+(e) = A \cdot F_-(e)
$$
Where:
	•	F_-(e) is a solution to the homogeneous ODE (\mathcal{L} - \rho)f = 0,
	•	It’s defined as:
$$	
F_-(e) = \int_0^\infty u^{\rho/\kappa - 1} \exp\left(-\frac{1}{2}\sigma^2 u + \kappa(\theta - e)u\right) du
$$
Take derivative:
$$
G^+’(e) = A \cdot F_-’(e)
$$
⸻

🧩 Now: Apply Matching Conditions at e = E^*_{\text{entry}}

From value matching:
$$
G^+(e) = H^+(e) - e - c = A \cdot F_-(e)
$$
From smooth pasting:
$$
G^+’(e) = H^+’(e) - 1 = A \cdot F_-’(e)
$$
Now eliminate A by dividing the equations:
$$
\frac{F_-(e)}{F_-’(e)} = \frac{H^+(e) - e - c}{H^+’(e) - 1}
$$
✅ This is the entry threshold equation that must be solved numerically.

⸻

🧠 Intuition
	•	Left-hand side: describes the value of waiting, governed by the OU process dynamics.
	•	Right-hand side: describes the gain from entering now.

You solve for e where these two forces exactly balance — this gives you the optimal entry level.

⸻


In [72]:
#using AD fuller test to check for cointegration


import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Download data (example: SPY vs. XLE)
tickers = ["GOOGL", "NVDA"]
data = yf.download(tickers, start="2020-01-01", end="2025-12-31")["Close"]
data = data.dropna()

# ticker=['googl', 'aal']
# tickers=[tik.upper() for tik in ticker]
# data = price_data[tickers]


pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
np.set_printoptions(threshold=np.inf)


data.index = pd.to_datetime(data.index)
#data = data.asfreq('min')  # 'T' = 1-minute frequency
data = data.dropna()

# Cointegration test using Engle-Granger 2-step method
def check_cointegration(y, x):
    model = sm.OLS(y, sm.add_constant(x)).fit()
    spread = y - model.predict(sm.add_constant(x))  # more robust
    adf_stat, p_value, _, _, crit_values, _ = adfuller(spread)
    
    print("ADF Statistic:", round(adf_stat, 4))
    print("ADF p-value:", round(p_value, 4))
    print("Critical Values:", crit_values)
    
    if p_value < 0.05:
        print("✅ Residuals are stationary ⇒ series are cointegrated.")
    else:
        print("❌ Residuals are non-stationary ⇒ no cointegration.")
    
    return spread, p_value

# Run the test
spread, p_value = check_cointegration(data[tickers[0]], data[tickers[1]])

print(data.tail(5))

[*********************100%***********************]  2 of 2 completed

ADF Statistic: -2.5646
ADF p-value: 0.1005
Critical Values: {'1%': np.float64(-3.435170967430817), '5%': np.float64(-2.8636690928667523), '10%': np.float64(-2.5679035297726274)}
❌ Residuals are non-stationary ⇒ no cointegration.
Ticker           GOOGL        NVDA
Date                              
2025-05-23  168.470001  131.289993
2025-05-27  172.899994  135.500000
2025-05-28  172.360001  134.809998
2025-05-29  171.860001  139.190002
2025-05-30  171.740005  135.130005


In [73]:
# OU estimator is dynamically estimated using a rolling window of 30 days.

import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from scipy.integrate import quad
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Step 1: Download historical data
tickers = ["GOOGL", "NVDA"]
data = yf.download(tickers, start="2020-01-01", end="2025-12-25")["Close"]
data.dropna(inplace=True)

# Step 2: Construct spread using OLS
X = sm.add_constant(data[tickers[1]])
model = OLS(data[tickers[0]], X).fit()
spread = data[tickers[0]] - model.predict(X)
# spead is the residual: actual appl price - predicted appl price
print(model.summary())


# Step 3: Estimate OU parameters, adjusting window size

def estimate_ou_params(spread, window=10):
    params = pd.DataFrame(index=spread.index, columns=["kappa", "theta", "sigma", "half_life"])
    dt = 1 / 252  # daily frequency
    for i in range(window, len(spread)):
        current_spread = spread.iloc[i - window:i]
        X_t = current_spread[:-1].values
        dX = np.diff(current_spread.values)
        model = sm.OLS(dX, sm.add_constant(X_t)).fit()
        a, b = model.params
        resid_std = np.std(model.resid, ddof=1)
        kappa = -b
        theta = -a / b if b != 0 else np.mean(current_spread)
        sigma = resid_std / np.sqrt(dt)
        half_life = np.log(2) / kappa if kappa > 0 else np.inf
        params.iloc[i] = [kappa, theta, sigma, half_life]
    return params.dropna()

ou_params = estimate_ou_params(spread)

# --- Step 4: Combine OU params and compute Z-score ---
spread_df = pd.DataFrame({"Spread": spread})
spread_df = spread_df.merge(ou_params, left_index=True, right_index=True, how="inner")
spread_df["ZScore"] = (spread_df["Spread"] - spread_df["Spread"].rolling(30).mean()) / spread_df["Spread"].rolling(30).std()
spread_df.dropna(inplace=True)

# --- Step 5: Define F_minus and F_minus_prime with safety ---
def safe_exp(x):
    return np.exp(np.clip(x, -700, 700))  # prevent overflow

def F_minus(e, kappa, theta, sigma, rho, upper_limit=20):
    exponent = rho / kappa - 1
    integrand = lambda u: u**max(exponent, -50) * safe_exp(-0.5 * sigma**2 * u + kappa * (theta - e) * u)
    result, _ = quad(integrand, 0, upper_limit, limit=100)
    return result

def F_minus_prime(e, kappa, theta, sigma, rho, upper_limit=20):
    exponent = rho / kappa
    integrand = lambda u: -kappa * u**max(exponent, -50) * safe_exp(-0.5 * sigma**2 * u + kappa * (theta - e) * u)
    result, _ = quad(integrand, 0, upper_limit, limit=100)
    return result

# --- Step 6: Compute F_minus and F_minus_prime over time ---
rho = 0.1  # discount rate

f_vals = []
f_prime_vals = []

for idx, row in spread_df.iterrows():
    try:
        f_val = F_minus(row["Spread"], row["kappa"], row["theta"], row["sigma"], rho)
        f_prime_val = F_minus_prime(row["Spread"], row["kappa"], row["theta"], row["sigma"], rho)
    except:
        f_val, f_prime_val = np.nan, np.nan
    f_vals.append(f_val)
    f_prime_vals.append(f_prime_val)

spread_df["F_minus"] = f_vals
spread_df["F_minus_prime"] = f_prime_vals

# Step 7: Define transaction cost and dynamic exit level
c = 0.5  # example transaction cost
spread_df["E_exit_star"] = spread_df["Spread"].rolling(60).max() * 0.8  # dynamic heuristic exit level (e.g., 80% of recent max)

# Step 8: Define dynamic H_exit and H_exit_prime functions per row

def compute_H_exit(e, e_star, kappa, c):
    if e >= e_star:
        return e - c
    else:
        return (e_star - c) * np.exp(kappa * (e - e_star)) + c

def compute_H_exit_prime(e, e_star, kappa, c):
    if e >= e_star:
        return 1.0
    else:
        return (e_star - c) * kappa * np.exp(kappa * (e - e_star))

# Step 9: Apply H_exit and H_exit_prime across time using OU params
H_vals = []
H_prime_vals = []

for idx, row in spread_df.iterrows():
    try:
        e = row["Spread"]
        e_star = row["E_exit_star"]
        kappa = row["kappa"]
        h_val = compute_H_exit(e, e_star, kappa, c)
        h_prime_val = compute_H_exit_prime(e, e_star, kappa, c)
    except:
        h_val, h_prime_val = np.nan, np.nan
    H_vals.append(h_val)
    H_prime_vals.append(h_prime_val)

spread_df["H_exit"] = H_vals
spread_df["H_exit_prime"] = H_prime_vals

# Step 10: Solve for entry threshold using root_scalar
from scipy.optimize import root_scalar

entry_thresholds = []

for idx, row in spread_df.iterrows():
    try:
        e_star = row["E_exit_star"]
        kappa = row["kappa"]
        theta = row["theta"]
        sigma = row["sigma"]
        
        # Skip invalid rows
        if pd.isnull([e_star, kappa, theta, sigma]).any() or kappa <= 0 or sigma <= 0:
            entry_thresholds.append(np.nan)
            continue
        
        # Local F_minus and F_minus_prime functions using that day's params
        def Fm(e):
            exponent = rho / kappa - 1
            integrand = lambda u: u**max(exponent, -50) * np.exp(np.clip(-0.5 * sigma**2 * u + kappa * (theta - e) * u, -700, 700))
            result, _ = quad(integrand, 0, 20, limit=100)
            return result

        def Fm_prime(e):
            exponent = rho / kappa
            integrand = lambda u: -kappa * u**max(exponent, -50) * np.exp(np.clip(-0.5 * sigma**2 * u + kappa * (theta - e) * u, -700, 700))
            result, _ = quad(integrand, 0, 20, limit=100)
            return result
        
        # H_exit and derivative for this specific day
        def H(e):
            if e >= e_star:
                return e - c
            else:
                return (e_star - c) * np.exp(kappa * (e - e_star)) + c

        def H_prime(e):
            if e >= e_star:
                return 1.0
            else:
                return (e_star - c) * kappa * np.exp(kappa * (e - e_star))
        
        # Entry threshold equation
        def entry_eq(E):
            lhs = Fm(E) / Fm_prime(E)
            rhs_num = H(E) - E - c
            rhs_denom = H_prime(E) - 1
            if rhs_denom == 0:
                return np.inf
            return lhs - (rhs_num / rhs_denom)
        
        # Try solving root within valid domain
        bracket_low = spread.min()
        bracket_high = e_star - 0.01
        
        if bracket_high <= bracket_low:
            entry_thresholds.append(np.nan)
            continue
        
        sol = root_scalar(entry_eq, bracket=[bracket_low, bracket_high], method='brentq')
        entry_thresholds.append(sol.root if sol.converged else np.nan)

    except Exception as e:
        entry_thresholds.append(np.nan)

spread_df["E_entry_star"] = entry_thresholds
spread_df["EntrySignal"] = spread_df["Spread"] < spread_df["E_entry_star"]

# This loop uses the OU parameters per day to compute E_entry_star.
# It gives you a dynamic entry threshold for every time index (row in spread_df) where the OU estimation is valid.
# You can now use Spread < E_entry_star as a buy signal.

# Step 11: Simulate trading using dynamic thresholds and OU signals
position = 0
entry_price = 0
initial_cash = 10000
cash = initial_cash
pnl_history = []
trade_log = []

for date, row in spread_df.iterrows():
    e = row["Spread"]
    z = row["ZScore"]
    e_entry_star = row.get("E_entry_star", np.nan)
    e_exit_star = row.get("E_exit_star", np.nan)

    if pd.notna(e_entry_star) and pd.notna(e_exit_star):
        if position == 0 and e <= e_entry_star and z < -1.0:
            # Enter trade
            position = 1
            entry_price = e
            entry_date = date
            trade_log.append([entry_date.strftime("%Y-%m-%d"), "", "ENTER", round(e, 2), ""])

        elif position == 1 and e >= e_exit_star and z > 0.5:
            # Exit trade
            position = 0
            profit = e - entry_price
            cash += profit
            exit_date = date
            trade_log[-1][1] = exit_date.strftime("%Y-%m-%d")
            trade_log[-1][-1] = round(profit, 2)

    # Mark-to-market P&L
    if position == 0:
        pnl_history.append(cash)
    else:
        pnl_history.append(cash + (e - entry_price))

# Store final P&L and trade logs
spread_df["PnL"] = pnl_history
final_value = pnl_history[-1]
total_return = (final_value - initial_cash) / initial_cash * 100

print(f"Final Portfolio Value: ${final_value:.2f}")
print(f"Total Return: {total_return:.2f}%")

trade_df = pd.DataFrame(trade_log, columns=["EntryDate", "ExitDate", "Action", "EntrySpread", "Profit"])

# Create a three-row subplot (Prices, Spread, P&L)
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    subplot_titles=(
        f"{tickers[0]} and {tickers[1]} Prices",
        "Spread and Trade Points with Z-Score Filter",
        f"Cumulative Profit & Loss (Final Value: ${final_value:,.2f})"
    )
)

# --- Row 1: Price Plot ---
fig.add_trace(
    go.Scatter(
        x=data.index, y=data[tickers[0]],
        mode="lines", name=tickers[0], line=dict(color="blue"),
        hovertemplate="%{x|%b %d} — " + tickers[0] + ": %{y:.2f}<extra></extra>"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=data.index, y=data[tickers[1]],
        mode="lines", name=tickers[1], line=dict(color="orange"),
        hovertemplate="%{x|%b %d} — " + tickers[1] + ": %{y:.2f}<extra></extra>"
    ),
    row=1, col=1
)

# --- Row 2: Spread Plot with Dynamic Entry/Exit Thresholds and Z-Score ---
fig.add_trace(
    go.Scatter(
        x=spread_df.index, y=spread_df["Spread"],
        mode="lines", name="Spread", line=dict(color="blue"),
        hovertemplate="%{x|%b %d} — Spread: %{y:.2f}<extra></extra>"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=spread_df.index, y=spread_df["E_entry_star"],
        mode="lines", name="Dynamic Entry Threshold",
        line=dict(dash="dash", color="green"),
        hovertemplate="%{x|%b %d} — Entry*: %{y:.2f}<extra></extra>"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=spread_df.index, y=spread_df["E_exit_star"],
        mode="lines", name="Dynamic Exit Threshold",
        line=dict(dash="dash", color="red"),
        hovertemplate="%{x|%b %d} — Exit*: %{y:.2f}<extra></extra>"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=spread_df.index, y=spread_df["ZScore"],
        mode="lines", name="Z-Score",
        line=dict(color="purple", dash="dot"),
        hovertemplate="%{x|%b %d} — Z: %{y:.2f}<extra></extra>",
        yaxis="y2"  # Reuse primary y-axis for simplicity or define a secondary one
    ),
    row=2, col=1
)
# --- Row 3: Cumulative P&L Plot ---
fig.add_trace(
    go.Scatter(
        x=spread_df.index, y=pnl_history,
        mode="lines", name="Cumulative P&L", line=dict(color="black"),
        hovertemplate="%{x|%b %d} — P&L: %{y:,.2f}<extra></extra>"
    ),
    row=3, col=1
)

# --- Layout ---
fig.update_layout(
    height=900,
    title_text="Pairs Trading Analysis: Price, Spread, and P&L",
    xaxis=dict(title="Date", tickformat="%b %d"),
    xaxis2=dict(title="Date", tickformat="%b %d"),
    xaxis3=dict(title="Date", tickformat="%b %d"),
    yaxis=dict(title=f"{tickers[0]} / {tickers[1]} Price"),
    yaxis2=dict(title="Spread"),
    yaxis3=dict(title="Account Value"),
    showlegend=True
)

# Add trade entry/exit markers to Spread Plot
for trade in trade_log:
    entry_date = trade[0]
    exit_date = trade[1]
    entry_val = spread_df.loc[entry_date]["Spread"] if entry_date in spread_df.index else None
    exit_val = spread_df.loc[exit_date]["Spread"] if exit_date in spread_df.index else None

    if entry_val:
        fig.add_trace(go.Scatter(
            x=[entry_date], y=[entry_val],
            mode="markers+text", name="Trade Entry",
            marker=dict(symbol="triangle-up", color="green", size=10),
            text=["Entry"], textposition="top center"
        ), row=2, col=1)

    if exit_val:
        fig.add_trace(go.Scatter(
            x=[exit_date], y=[exit_val],
            mode="markers+text", name="Trade Exit",
            marker=dict(symbol="triangle-down", color="red", size=10),
            text=["Exit"], textposition="bottom center"
        ), row=2, col=1)

fig.show(renderer="browser")

# Step 10: Print trade log
print("Trade Log:\n")
print(f"{'Entry Date':<12} {'Exit Date':<12} {'Action':<8} {'Spread':>8} {'Profit':>10}")
for trade in trade_log:
    print(f"{trade[0]:<12} {trade[1]:<12} {trade[2]:<8} {trade[3]:>8.3f} {str(trade[4]):>10}")

# --- Output Preview ---
print(spread_df[["E_entry_star", "EntrySignal", "half_life", "Spread"]].tail(20)) # Show last 20 rows of the spread DataFrame with OU params and Z-scores
print(trade_log)

[*********************100%***********************]  2 of 2 completed
/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_38667/1382386998.py:61: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_38667/1382386998.py:61: IntegrationWarning:

The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.

/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_38667/1382386998.py:67: IntegrationWarning:

The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.



                            OLS Regression Results                            
Dep. Variable:                  GOOGL   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                     3479.
Date:                Sat, 31 May 2025   Prob (F-statistic):               0.00
Time:                        16:25:56   Log-Likelihood:                -5880.0
No. Observations:                1360   AIC:                         1.176e+04
Df Residuals:                    1358   BIC:                         1.177e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         92.2087      0.715    129.048      0.0

/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_38667/1382386998.py:67: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_38667/1382386998.py:152: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_38667/1382386998.py:146: IntegrationWarning:

The maximum number of subdivisions (100) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.

/var/folders/zv/byjsmvy90d57p6xyyc10dt240000gn/T/ipykernel_38667/1382386998.py:146: IntegrationWarning:

The integral is probably

Final Portfolio Value: $10000.00
Total Return: 0.00%
Trade Log:

Entry Date   Exit Date    Action     Spread     Profit
            E_entry_star  EntrySignal half_life     Spread
Date                                                      
2025-05-02           NaN        False  2.011671  -5.887292
2025-05-05           NaN        False  1.145188  -5.245783
2025-05-06           NaN        False  1.025476  -6.035765
2025-05-07           NaN        False  0.898937 -20.274698
2025-05-08           NaN        False  3.000985 -17.585098
2025-05-09           NaN        False  2.148456 -18.626448
2025-05-12           NaN        False  2.726304 -17.226041
2025-05-13           NaN        False  2.380586 -20.859278
2025-05-14           NaN        False  2.894785 -18.690928
2025-05-15           NaN        False  2.183084 -19.754795
2025-05-16           NaN        False   1.76996 -17.911640
2025-05-19           NaN        False  1.148492 -17.677033
2025-05-20           NaN        False  0.600762 -19.42